<a href="https://colab.research.google.com/github/bpizarrogalleguillos/IAmetodologia2025/blob/main/data_code_supervisado_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [ ]:
#@title 📊 Crear un dataset clínico simulado
# Simulación de datos clínicos de pacientes
np.random.seed(42)
n = 300

# Simular fiebre primero
fiebre = np.random.choice([0, 1], size=n, p=[0.3, 0.7])

# Leucocitosis más alta si hay fiebre
leucocitosis = np.random.normal(loc=9500 + 3000*fiebre, scale=1500, size=n).astype(int)

# PCR más alta si hay fiebre y leucocitosis alta
pcr = 10 + 0.002 * leucocitosis + 10 * fiebre + np.random.normal(0, 5, size=n)
pcr = np.round(pcr, 1)

# Simular otras variables clínicas y de imagen
sexo = np.random.choice(['M', 'F'], size=n)
tos = np.random.choice([0, 1], size=n, p=[0.4, 0.6])
infiltrado_rx = np.random.choice([0, 1], size=n, p=[0.5, 0.5])
engrosamiento_parietal = np.random.choice([0, 1], size=n, p=[0.7, 0.3])
foco_condensacion = np.random.choice([0, 1], size=n, p=[0.6, 0.4])
patron_intersticial = np.random.choice([0, 1], size=n, p=[0.65, 0.35])
derrame_pleural = np.random.choice([0, 1], size=n, p=[0.8, 0.2])
broncograma_aereo = np.random.choice([0, 1], size=n, p=[0.7, 0.3])
cruza_cisuras = np.random.choice([0, 1], size=n, p=[0.85, 0.15])
edad = np.random.randint(18, 90, size=n)

# Crear el DataFrame
data = pd.DataFrame({
    'edad': edad,
    'sexo': sexo,
    'fiebre': fiebre,
    'tos': tos,
    'infiltrado_rx': infiltrado_rx,
    'engrosamiento_parietal': engrosamiento_parietal,
    'foco_condensacion': foco_condensacion,
    'patron_intersticial': patron_intersticial,
    'derrame_pleural': derrame_pleural,
    'broncograma_aereo': broncograma_aereo,
    'cruza_cisuras': cruza_cisuras,
    'leucocitosis': leucocitosis,
    'pcr': pcr
})

# Diagnóstico de neumonía más influenciado por fiebre, leucocitosis, pcr, condensación, tos y broncograma
# Diagnóstico de neumonía con mayor peso para foco de condensación
prob_neumonia = (
    0.05 +
    0.2 * data['fiebre'] +
    0.2 * (data['leucocitosis'] > 12000).astype(int) +
    0.2 * (data['pcr'] > 30).astype(int) +
    0.25 * data['foco_condensacion'] +  # antes 0.15
    0.15 * data['tos'] +
    0.15 * data['broncograma_aereo']
)
prob_neumonia = np.clip(prob_neumonia, 0, 0.95)
data['diagnostico_neumonia'] = np.random.binomial(1, prob_neumonia)

# Ingreso a UCI
prob_uci = (
    0.05 +
    0.3 * data['diagnostico_neumonia'] +
    0.2 * (data['edad'] > 70).astype(int) +
    0.2 * (data['pcr'] > 40).astype(int) +
    0.2 * data['derrame_pleural'] +
    0.1 * (data['leucocitosis'] > 14000).astype(int)
)
prob_uci = np.clip(prob_uci, 0, 0.9)
data['ingreso_uci'] = np.random.binomial(1, prob_uci)

# Intubación
prob_intubacion = (
    0.03 +
    0.4 * data['ingreso_uci'] +
    0.2 * data['diagnostico_neumonia'] +
    0.2 * data['broncograma_aereo'] +
    0.1 * (data['edad'] > 75).astype(int)
)
prob_intubacion = np.clip(prob_intubacion, 0, 0.8)
data['intubacion'] = np.random.binomial(1, prob_intubacion)

# Mortalidad
prob_mortalidad = (
    0.02 +
    0.3 * data['intubacion'] +
    0.2 * data['ingreso_uci'] +
    0.2 * (data['edad'] > 80).astype(int) +
    0.2 * (data['pcr'] > 50).astype(int)
)
prob_mortalidad = np.clip(prob_mortalidad, 0, 0.6)
data['mortalidad'] = np.random.binomial(1, prob_mortalidad)

# Mostrar primeras filas
data.head()


In [ ]:
data.to_csv('data.csv', index=False)

Dataset 2

In [3]:
# 📊 Crear un dataset clínico simulado
import numpy as np
import pandas as pd

np.random.seed(42)
n = 300

# Simular fiebre primero
fiebre = np.random.choice([0, 1], size=n, p=[0.3, 0.7])

# Leucocitosis más alta si hay fiebre
leucocitosis = np.random.normal(loc=9500 + 3000 * fiebre, scale=1500, size=n).astype(int)

# PCR más alta si hay fiebre y leucocitosis alta
pcr = 10 + 0.002 * leucocitosis + 10 * fiebre + np.random.normal(0, 5, size=n)
pcr = np.round(pcr, 1)

# Simular otras variables clínicas y de imagen
sexo = np.random.choice(['M', 'F'], size=n)
tos = np.random.choice([0, 1], size=n, p=[0.4, 0.6])
infiltrado_rx = np.random.choice([0, 1], size=n, p=[0.5, 0.5])
engrosamiento_parietal = np.random.choice([0, 1], size=n, p=[0.7, 0.3])
foco_condensacion = np.random.choice([0, 1], size=n, p=[0.6, 0.4])
patron_intersticial = np.random.choice([0, 1], size=n, p=[0.65, 0.35])
derrame_pleural = np.random.choice([0, 1], size=n, p=[0.8, 0.2])
broncograma_aereo = np.random.choice([0, 1], size=n, p=[0.7, 0.3])
cruza_cisuras = np.random.choice([0, 1], size=n, p=[0.85, 0.15])
edad = np.random.randint(18, 90, size=n)

# Temperatura corporal según fiebre
temperatura = np.where(
    fiebre == 1,
    np.random.normal(loc=38.5, scale=0.5, size=n),  # fiebre
    np.random.normal(loc=36.8, scale=0.3, size=n)   # sin fiebre
)
temperatura = np.clip(temperatura, 36.5, 41.0)
temperatura = np.round(temperatura, 1)

# Crear el DataFrame
data = pd.DataFrame({
    'edad': edad,
    'sexo': sexo,
    'fiebre': fiebre,
    'temperatura': temperatura,
    'tos': tos,
    'infiltrado_rx': infiltrado_rx,
    'engrosamiento_parietal': engrosamiento_parietal,
    'foco_condensacion': foco_condensacion,
    'patron_intersticial': patron_intersticial,
    'derrame_pleural': derrame_pleural,
    'broncograma_aereo': broncograma_aereo,
    'cruza_cisuras': cruza_cisuras,
    'leucocitosis': leucocitosis,
    'pcr': pcr
})

# Índice radiológico (0 a 10) basado en hallazgos de imagen
data['indice_rx'] = (
    3.0 * data['foco_condensacion'] +
    2.0 * data['broncograma_aereo'] +
    1.5 * data['derrame_pleural'] +
    1.5 * data['infiltrado_rx'] +
    1.0 * data['engrosamiento_parietal'] +
    0.5 * data['patron_intersticial'] +
    0.5 * data['cruza_cisuras'] +
    1.5 * data['fiebre'] +
    0.05 * data['pcr'] +                 # ponderación ajustada por escala
    0.0002 * data['leucocitosis']        # ponderación ajustada por escala
).clip(upper=20)

# Diagnóstico de neumonía
prob_neumonia = (
    0.05 +
    0.2 * data['fiebre'] +
    0.2 * (data['leucocitosis'] > 12000).astype(int) +
    0.2 * (data['pcr'] > 30).astype(int) +
    0.25 * data['foco_condensacion'] +
    0.15 * data['tos'] +
    0.15 * data['broncograma_aereo']
)
prob_neumonia = np.clip(prob_neumonia, 0, 0.95)
data['diagnostico_neumonia'] = np.random.binomial(1, prob_neumonia)

# Ingreso a UCI
prob_uci = (
    0.05 +
    0.3 * data['diagnostico_neumonia'] +
    0.2 * (data['edad'] > 70).astype(int) +
    0.2 * (data['pcr'] > 40).astype(int) +
    0.2 * data['derrame_pleural'] +
    0.1 * (data['leucocitosis'] > 14000).astype(int)
)
prob_uci = np.clip(prob_uci, 0, 0.9)
data['ingreso_uci'] = np.random.binomial(1, prob_uci)

# Intubación
prob_intubacion = (
    0.03 +
    0.4 * data['ingreso_uci'] +
    0.2 * data['diagnostico_neumonia'] +
    0.2 * data['broncograma_aereo'] +
    0.1 * (data['edad'] > 75).astype(int)
)
prob_intubacion = np.clip(prob_intubacion, 0, 0.8)
data['intubacion'] = np.random.binomial(1, prob_intubacion)

# Mortalidad
prob_mortalidad = (
    0.02 +
    0.3 * data['intubacion'] +
    0.2 * data['ingreso_uci'] +
    0.2 * (data['edad'] > 80).astype(int) +
    0.2 * (data['pcr'] > 50).astype(int)
)
prob_mortalidad = np.clip(prob_mortalidad, 0, 0.6)
data['mortalidad'] = np.random.binomial(1, prob_mortalidad)

# Mostrar primeras filas
print(data.head())

   edad sexo  fiebre  temperatura  tos  infiltrado_rx  engrosamiento_parietal  \
0    58    F       1         39.0    1              0                       0   
1    48    F       1         38.4    0              1                       0   
2    39    F       1         38.6    1              0                       0   
3    21    M       1         38.3    1              0                       1   
4    30    M       0         37.1    1              0                       0   

   foco_condensacion  patron_intersticial  derrame_pleural  broncograma_aereo  \
0                  1                    0                1                  0   
1                  0                    0                0                  0   
2                  1                    1                0                  0   
3                  0                    0                0                  0   
4                  1                    0                0                  0   

   cruza_cisuras  leucocit

In [4]:
data.to_csv('data_2.csv', index=False)